In [1]:
from riotwatcher import RiotWatcher
import json

In [2]:
import sys

In [3]:
watcher = RiotWatcher('RGAPI-KEY-GOES-HERE')

In [ ]:
# matches is a list of (game_id, match_starting_time, region, gamemode, match_champions_id)
matches = []
match_champions = []
# dict of <summoner name, region>
ids_from_featured_games = {}

match_id = []
participants_list = []

In [ ]:
featured_games = watcher.spectator.featured_games('na1')

for game in featured_games['gameList']:
    # get the region of the match, e.g. NA1
    region = game['platformId']
    # write in the record dict the id of the match, starting time, platform ID (region)
    # and game mode
    record = [game['gameId'], game['gameStartTime'],
              game['platformId'], game['gameMode']]
    # for each participant of the current match
    for participants in game['participants']:
        # write in the ids_from_featured_games dict the summoner name as key, and region as value
        ids_from_featured_games[participants['summonerName']] = region
        # append champion id to record
        record.append(participants['championId'])
    matches.append(record)

In [ ]:
for name, region in ids_from_featured_games.iteritems():
    # retrieve summoner info
    summoner = watcher.summoner.by_name(region, name)
    user_matches = watcher.match.matchlist_by_account(
        region, summoner['accountId'])
    # matches of user
    for m in user_matches['matches']:
        # get match info
        game = watcher.match.by_id(region, m['gameId'])
        record = [game['seasonId'], game['gameId'], game['gameCreation'],
                  game['platformId'], game['gameMode']]
        # for each player on this match
        for participant in game['participantIdentities']:
            # append the summoner id
            participants_list.append(participant['player']['accountId'])
            if len(participants_list)%50 == 0:
                print 'participants: {}'.format(len(participants_list))

In [ ]:
file = open('participants.txt', 'a')
for p in participants_list:
  file.write("%s\n" % p)

In [4]:
with open('participants.txt') as f:
    all_participants = f.read().splitlines()

In [5]:
game_ids = []

In [6]:
# retrieve the matches id from the participant IDs
game_ids = []
# load all the participants from the list
for p in all_participants:
    try:
        p_matches = watcher.match.matchlist_by_account(account_id = p, region = 'NA1')
    except:
        continue
    for m in p_matches['matches']:
        game_ids.append(m['gameId'])   

In [ ]:
file = open('gameids.txt', 'a')
for p in game_ids:
  file.write("%s\n" % p)

In [9]:
with open('gameids.txt') as f:
    all_game_ids = f.read().splitlines()

In [11]:
match_data = []
for gid in all_game_ids:
    try:
        # get match information
        m = watcher.match.by_id(match_id = gid, region = 'na1')
    except:
        print 'exception match {}'.format(gid)
        continue
    try:
        row = (m['seasonId'], m['queueId'], m['gameId'], m['gameVersion'], m['platformId'], m['gameMode'], m['mapId'], m['gameType'], 
            m['participants'][0]['championId'], #the first 5 champions are from team 1
            m['participants'][1]['championId'],
            m['participants'][2]['championId'],
            m['participants'][3]['championId'],
            m['participants'][4]['championId'],
            m['participants'][5]['championId'],
            m['participants'][6]['championId'],
            m['participants'][7]['championId'],
            m['participants'][8]['championId'],
            m['participants'][9]['championId'],
            m['gameDuration'], m['gameCreation'],
            m['teams'][0]['win'], #outcome team 1
            m['teams'][1]['win']) #outcome team 2
    except:
        print (sys.exc_info()[0])
        continue
    match_data.append(row)

In [ ]:
file = open('match_data.txt', 'a')
for p in match_data:
    # convert each line into a comma separated string so it is easier to read as a CSV
    file.write("{}\n".format(','.join(map(str, p))))